In [1]:
!pip install transliterate -q

In [ ]:
!pip install nltk -q

In [92]:
!pip install -U -q sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Matching товаров ООО "ПРОСЕПТ"

## Введение

**ООО «ПРОСЕПТ»** — российская производственная компания, специализирующаяся
на выпуске профессиональной химии. В своей работе используют опыт ведущих
мировых производителей и сырье крупнейших химических концернов. Производство и
логистический центр расположены в непосредственной близости от Санкт-Петербурга,
откуда продукция компании поставляется во все регионы России.
Сайт: https://prosept.ru/


**Введение в задачу**:

Заказчик производит несколько сотен различных товаров бытовой и промышленной
химии, а затем продаёт эти товары через дилеров. Дилеры, в свою очередь,
занимаются розничной продажей товаров в крупных сетях магазинов и на онлайн
площадках.

Для оценки ситуации, управления ценами и бизнесом в целом, заказчик
периодически собирает информацию о том, как дилеры продают их товар. Для этого
они парсят сайты дилеров, а затем сопоставляют товары и цены.
Зачастую описание товаров на сайтах дилеров отличаются от того описания, что даёт
заказчик. Например, могут добавляться новый слова (“универсальный”,
“эффективный”), объём (0.6 л -> 600 мл). Поэтому сопоставление товаров дилеров с
товарами производителя делается вручную.
Цель этого проекта - разработка решения, которое отчасти автоматизирует процесс
сопоставления товаров. Основная идея - предлагать несколько товаров заказчика,
которые с наибольшей вероятностью соответствуют размечаемому товару дилера.
Предлагается реализовать это решение, как онлайн сервис, открываемый в веб-
браузере. Выбор наиболее вероятных подсказок делается методами машинного
обучения.

**Документация к предоставленным данным**:

Заказчик предоставил несколько таблиц (дамп БД), содержащих необходимые
данные:

1 marketing_dealer - список дилеров;

2 marketing_dealerprice - результат работы парсера площадок дилеров:

- product_key - уникальный номер позиции;

- price - цена;

- product_url - адрес страницы, откуда собраны данные;

- product_name - заголовок продаваемого товара;

- date - дата получения информации;

- dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)


3 marketing_product - список товаров, которые производит и распространяет
заказчик;

- article - артикул товара;

- ean_13 - код товара (см. EAN 13)

- name - название товара;

- cost - стоимость;

- min_recommended_price - рекомендованная минимальная цена;

- recommended_price - рекомендованная цена;

- category_id - категория товара;

- ozon_name - названиет товара на Озоне;

- name_1c - название товара в 1C;

- wb_name - название товара на Wildberries;

- ozon_article - описание для Озон;

- wb_article - артикул для Wildberries;

- ym_article - артикул для Яндекс.Маркета;

4 marketing_productdealerkey - таблица матчинга товаров заказчика и товаров
дилеров

- key - внешний ключ к marketing_dealerprice

- product_id - внешний ключ к marketing_product

- dealer_id - внешний ключ к marketing_dealer

## План работ


**До дедлайна 19:00 29 ноября**

1. Команда знакомится с предоставленными данными.

2. Формулируется DS задача, утверждается единые схема валидации решений и метрика качества.

3. Выбирается основной способ решения задачи (модель первого этапа с функционалом финального решения),
    который будет представлен к дедлайну 29 ноября:
    - Рассматриваются и валидируются разные способы предобработки входных данных модели.
    - Рассматриваются и валидируются разные ml движки решения.


4. Подготовка модели первого этапа к сдаче на ревью в том виде, в котором ею сможет пользоваться BackEnd департамент команды.

5. Подготовка репозитория решения с jupyter notebook, содержащим основные вехи разработки модели первого этапа.

**После дедлайна DS до единого дедлайна**

6. Генерация новых фичей, улучшение схемы предобработки входных данных.

7. Построение и валидация модели второго этапа (реранжирующий классификатор).

8. Тюнинг реранжирующего классификатора.

9. Предоставление финального решения BackEnd команде, помощь в его инициализации.

**10**. Оформление документации и ожидание результатов хакатона.


## Решение (молель первого этапа)

### Импорты

In [101]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import KFold

import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from transliterate import translit


from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

import torch
import torch.nn.functional as F
from torch import Tensor

### Загрузка данных

In [ ]:
!ls -lah

In [3]:
try:
    dealer = pd.read_csv('/kaggle/input/privat-matching/data/marketing_dealer.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    dealerprice = pd.read_csv('/kaggle/input/privat-matching/data/marketing_dealerprice.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    product = pd.read_csv('/kaggle/input/privat-matching/data/marketing_product.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    interactions = pd.read_csv('/kaggle/input/privat-matching/data/marketing_productdealerkey.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
except:
    dealer = pd.read_csv('marketing_dealer.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    dealerprice = pd.read_csv('marketing_dealerprice.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    product = pd.read_csv('marketing_product.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    interactions = pd.read_csv('marketing_productdealerkey.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    

### Знакомство с данными

**Таблица с результатами парсинга**

In [4]:
dealerprice

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2
...,...,...,...,...,...,...,...
20411,20566,534517451,823.0,https://www.ozon.ru/product/534517451,"Огнебиозащита для древесины PROSEPT 2 группа, ...",2023-07-31,18
20412,20567,530308963,418.0,https://www.ozon.ru/product/530308963,"Антисептик многофункциональный ФБС, ГОСТ 5 л",2023-07-31,18
20413,20568,531730388,2986.0,https://www.ozon.ru/product/531730388,"Средство для удаления ржавчины PROSEPT, 1 шт",2023-07-31,18
20414,20569,1090913025,579.0,https://www.ozon.ru/product/1090913025,Герметик акриловый межшовный для деревянных ко...,2023-07-31,18


In [5]:
dealerprice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20416 entries, 0 to 20415
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            20416 non-null  int64  
 1   product_key   20416 non-null  object 
 2   price         20416 non-null  float64
 3   product_url   20182 non-null  object 
 4   product_name  20416 non-null  object 
 5   date          20416 non-null  object 
 6   dealer_id     20416 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 1.1+ MB


In [6]:
dealerprice.head()

,id,product_key,price,product_url,product_name,date,dealer_id
0,2,546227,233.0,https://akson.ru//p/sredstvo_universalnoe_pros...,Средство универсальное Prosept Universal Spray...,2023-07-11,2
1,3,546408,175.0,https://akson.ru//p/kontsentrat_prosept_multip...,"Концентрат Prosept Multipower для мытья полов,...",2023-07-11,2
2,4,546234,285.0,https://akson.ru//p/sredstvo_dlya_chistki_lyus...,Средство для чистки люстр Prosept Universal An...,2023-07-11,2
3,5,651258,362.0,https://akson.ru//p/udalitel_rzhavchiny_prosep...,"Удалитель ржавчины PROSEPT RUST REMOVER 0,5л 0...",2023-07-11,2
4,6,546355,205.0,https://akson.ru//p/sredstvo_moyushchee_dlya_b...,Средство моющее для бани и сауны Prosept Multi...,2023-07-11,2


Почему-то product_key хранится как строки, хотя должен храниться как числа. Проблема какая-то.

**p.s.** Некоторые диллеры хранят ключи своих товаров как ссылки 

In [7]:
dealerprice.isna().sum()

id                0
product_key       0
price             0
product_url     234
product_name      0
date              0
dealer_id         0
dtype: int64

In [8]:
print(f"Уникальных диллеров в данных парсера: {len(dealerprice['dealer_id'].unique())}")

Уникальных диллеров в данных парсера: 18


In [9]:
prosept_count = dealerprice['product_name'].apply(lambda x: 1 if x.lower().find('prosept') != -1 else 0).copy().rename('prosept_count')
print(f"Название компании Prosept встречается в {prosept_count.sum()} названиях товаров ({prosept_count.mean() * 100:.2f}%)")

Название компании Prosept встречается в 12503 названиях товаров (61.24%)


**Таблица с информацией о продуктах заказчиков**

In [10]:
product.head(3)

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN


In [11]:
product.isna().sum()

Unnamed: 0             0
id                     0
article                0
ean_13                32
name                   2
cost                   5
recommended_price      5
category_id           49
ozon_name             38
name_1c               11
wb_name               41
ozon_article         131
wb_article           156
ym_article           159
wb_article_td        464
dtype: int64

In [12]:
product.duplicated().sum()

0

**Таблица с сопоставлением продуктов дилеров продуктам заказчиков**

In [13]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1700 entries, 0 to 1699
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1700 non-null   int64 
 1   key         1700 non-null   object
 2   dealer_id   1700 non-null   int64 
 3   product_id  1700 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 53.2+ KB


In [14]:
interactions.head(3)

,id,key,dealer_id,product_id
0,1,546227,2,12
1,2,651265,2,106
2,3,546257,2,200


In [15]:
interactions.isna().sum()

id            0
key           0
dealer_id     0
product_id    0
dtype: int64

In [16]:
interactions.duplicated().sum()

0

#### Предобработка грубых недостатков таблиц

*ссылки в ключах диллера*

**p.s.**  оказалось что не надо удалять

In [ ]:
# rows_indexes = dealerprice[dealerprice['product_key'].apply(lambda x: not x.strip().isdigit())].index
# dealerprice = dealerprice.drop(rows_indexes)
# del rows_indexes

*ссылки в ключах взаимодействий*

**p.s.**  оказалось что не надо удалять

In [ ]:
# rows_indexes = interactions[interactions['key'].apply(lambda x: not x.strip().isdigit())].index
# interactions = interactions.drop(rows_indexes)
# del rows_indexes

In [ ]:
# interactions = interactions.reset_index(drop=True)

*пропуски в title PRODUCT* 

In [17]:
product[product['name'].isna()]

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
23,23,503,0024-7 о,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150126213.0,NaN,NaN
35,35,504,w022-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
product = product.dropna(subset='name')

In [19]:
product = product.reset_index(drop=True)

*дубли в id диллеров*

**p.s.** Ключи продуктов уникальные лишь в рамках отдельного диллера, так что удаляем дубликаты по двум столбцам. Чтобы в датасете остались только актуальные записи, перед удалением дублей сортируем таблицу по дате.

In [20]:
dealerprice = dealerprice.sort_values('date', ascending=False).drop_duplicates(subset=['product_key', 'dealer_id'])

In [21]:
dealerprice = dealerprice.reset_index(drop=True)

#### Функции чтения и первичной обработки входящих сырых таблиц

Функция чтения  **marketing_dealerprice**:

In [112]:
def dealerprice_table(table_path='marketing_dealerprice.csv',
                      product_id_column='product_key',
                      dealer_id_column='dealer_id',
                      read_params={'on_bad_lines': "skip",
                                   'encoding': 'utf-8',
                                   'sep': ';'}
                     ):
    '''
    Функция принимает:
    .Путь к csv файлу, содержащему результаты парсинга.
    .Названия колонок с id товаров и id дилеров
    .Параметры чтения csv можно указать, если вдруг они изменятся.
    '''
    
    table_csv = pd.read_csv(table_path, **read_params) 
    table_csv = table_csv.sort_values('date', ascending=False).drop_duplicates(subset=[product_id_column, dealer_id_column])
    
    return table_csv
    

Функция для чтения **marketing_product**

In [114]:
def prossept_products_table(table_path='marketing_product.csv',
                            product_names_column = 'name',
                            read_params={'on_bad_lines': "skip",
                                           'encoding': 'utf-8',
                                           'sep': ';'}
                             ):
    '''
    Функция принимает путь к csv файлу, содержащему актуальную информацию по товарам заказчика.
    Дополнительно указывается название колонки с внутренними неймингами для удаления плохих строк.
    '''
    
    table_csv = pd.read_csv(table_path, **read_params) 
    table_csv = table_csv.dropna(subset='name')
    
    return table_csv

**Текущие функции подвергнутся изменениям и улучшениям в процессе инициализации решения с бекендом в прод.**

### Формулирование DS задачи

#### Роль алгоритма в функционале приложения

Наш алгоритм должен помочь разметчику сопоставить товар диллера с одним из нескольких сотен товаров фирмы заказчика. Важно отметить, что финальное решение принимает именно разметчик. Так вот, насколько видит наша команда, алгоритм для каждого из предложенных товаров диллеров должен вернуть ранжированный список всех товаров заказчика так, чтобы релевантный айтем оказался максимально высоко в топе. Таким образом перед нами тривиальная задача ранжирования.

#### Особенности задачи

- Для товара диллера у заказчика есть только один релевантный айтем.
- Основными признаками для матчинга выступают нейминги товаров.
- Товары диллеров всегда новые, а множество товаров заказчика меняется редко.

#### Метрика качества

В качестве метрики качества мы утвердили Среднеобратный ранг (Mean Reciprocal Rank): $MRR = \frac{1}{N} \sum_{i=1}^{N} \frac{1}{\text{rank}_i}$ ,
где ${\text{rank}_i}$ это позиция релевантного айтема заказчика в ранжированном списке а ${N}$ это мощность множества товаров дилеров. Метрика выбрана по ряду причин, и по нашему общему мнению идеально совпадает с нашей задачей.

#### Схема валидации

Т.к. соответствия товаров диллеров товарам заказчика не изменяются со временем, обычная кросс-валидация идеально нам подходит. Будем валидироваться на 5 фолдах, в тестовой выборке каждого фолда будет примерно 300-400 соответствий.

#### Признаки айтемов 
**Первый этап**:
Как ранее указывалось, основным признаком для сопоставления являются нейминги товаров. Таким образом в качестве модели первого этапа мы представим функцию, помещающую все нейминги товаров заказчика в единое векторное пространство с помощью текстового векторизатора. Для формирования списка рекомендаций нейминг товара дилера помещается в то же пространство, а затем товары заказчика ранжируются по косинусной близости. В итоге уже обученная модель будет принимать массив с неймингами товаров дилера, а возвращать двумерный массив с рекомендациями.

**Второй этап**:
В качестве признаков модель второго этапа (классификатор) для каждой пары `товар_дилера - товар_заказчика` будет принимать вектора неймингов этих товаров, косинусное расстояние между векторами, а так же другие features какие мы нагенерируем. Далее по предсказанным вероятностям соответствия все айтемы заказчика реранжируются.

## Моделирование

Заглушка для бека

In [25]:
class PopularRecommender():

    def __init__(self, ):
        pass

    def fit(self,
            interactions,
            product_id='product_id'):
        
        self.recs = interactions[product_id].value_counts().index.tolist()

    def recommend(self,
                  dealer_ids: list[dict]):
        
        return np.array([self.recs for i in dealer_ids])
    
model = PopularRecommender()
model.fit(interactions)
model.recommend([1, 2, 3])

array([[400,  12, 249, ..., 189, 119, 440],
       [400,  12, 249, ..., 189, 119, 440],
       [400,  12, 249, ..., 189, 119, 440]])

### Модель первого этапа

Основным движком матчинга будет модель векторизации неймингов. На следующем этапе мы подтянем другие фичи для сопоставления. Например цены товаров. 

*функция предобработки неймингов для простых векторайзеров:*

In [24]:
ru_stop = stopwords.words('russian')
eng_stop = stopwords.words('english')
ru_stemmer = SnowballStemmer("russian")
eng_stemmer = SnowballStemmer("english")

def string_filter(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer):
    
    string = string.lower() 
    #убираем лишние символы
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    #добавляем пробелы между слитыми русскими и английскими словами вроде 'стенMultipower'
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    #добавляем пробелы между слитыми словами и числами '5литров' или '900ml'
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    #удаляем английские и русские стоп-слова и стемматизируем
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])

    return string

In [35]:
# Класс для ранжирования по дистанциям неймингов в едином векторном пространстве.
# Принимает модель векторизации текста с функционалом как у sklearn-овских векторайзеров,
# функцию подсчёта расстояния c функционалом sklearn-овских (в нашем решении работало только 
# косинусное), функцию предобработки сырых текстов (принимающую и возвращающую строку).

class DistanceRecommender():

    def __init__(self,
                 vectorizer,
                 simularity_func,
                 text_prep_func):
        
    #при инициализации указываем выбранный векторайзер (твой векторайзер должен вести себя как склёрновский)
    #Так же указываем функцию подсчёта расстояний. Т.к. мешки слов и мешки с TFIDF эт большие разреженные матрицы,
    #косинусное расстояние подходит хорошо. А, ну и вашу функцию предобработки неймингов тоже указать надо.
        
        self.vectorizer = vectorizer
        self.simularity_counter = simularity_func
        self.preprocessing = text_prep_func

    def fit(self,
            product_corpus,
            name_column,
            id_column):
    #принимает таблицу marketing_product, название колонок с названиями и айдишниками
        preprocessed_corpus = product_corpus[name_column].apply(self.preprocessing).values.tolist()
        
        self.vectorizer.fit(preprocessed_corpus)
        self.product_matrix = self.vectorizer.transform(preprocessed_corpus)
        self.product_index_to_id = {i: product_corpus.loc[i, id_column] for i in range(len(product_corpus))}
        
    def recommend(self,
                  dealer_corpus: list[dict]
                 ):
    #принимает непредобработанный корпус с названиями товаров которые над разметитить
    
        preprocessed_corpus = dealer_corpus.apply(self.preprocessing).values.tolist()
        vectors = self.vectorizer.transform(preprocessed_corpus)
        sims = self.simularity_counter(vectors, self.product_matrix)
        
        result = []
        for vec in sims:
            result += [[self.product_index_to_id[index] for index in vec.argsort()[::-1]]]
    # возвращает массив с ранжированными индексами товаров заказчика    
        return np.array(result)

Метрика MRR@10, для удобства своя реализация метрики Mean Reciprocal Rank ( https://rectools.readthedocs.io/en/latest/api/rectools.metrics.ranking.MRR.html ). Метрика идеально ложится на нашу задачу. Скалируется от 0 до 1, чем больше метрика тем выше релевантный айтем в списке рекоммендаций. 

In [28]:
def mean_reciprocal_rank(true_id,
                         recommendations,
                         k=10):
    
    reciprocal_ranks = []
    
    for i, rec in enumerate(recommendations):
        recs = rec[:k]
        relevant = true_id[i]
        
        if np.isin(relevant, recs):
            rank = np.where(recs == relevant)[0][0] + 1
            reciprocal_ranks += [1 / rank]
            
        else:
            reciprocal_ranks += [0]
            
    return np.mean(reciprocal_ranks)

In [30]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter)
}

Все модели первого этапа обучаются на корпусе неймингов заказчика:

In [31]:
for model in models.values():
    model.fit(product, 'name', 'id')

**кросс-валидация**

Валидировать решения будем на обычной кросс-валидации с 5 фолдами, ту же схему валидации будем использовать и для модели второго этапа. Все результаты будут сохраняться в единый фрейм:

In [32]:
kf = KFold(n_splits=5)

results_table = pd.DataFrame()
for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name])])

5it [00:02,  2.19it/s]
5it [00:02,  2.20it/s]
5it [00:02,  2.24it/s]
5it [00:02,  2.21it/s]
5it [00:02,  2.20it/s]
5it [00:02,  2.21it/s]
5it [00:02,  2.22it/s]
5it [00:02,  2.24it/s]
5it [00:02,  2.25it/s]
5it [00:02,  2.20it/s]
5it [00:02,  2.21it/s]
5it [00:02,  2.26it/s]


In [33]:
results_table.sort_values('MRR', ascending=False)

,MRR,std
CosRecTfIDF_1n,0.696339,0.037047
CosRecTfIDF_1n_2n,0.679717,0.038619
CosRecBow_1n,0.677707,0.045683
CosRecBow_1n_2n,0.676893,0.046476
CosRecBow_1n_3n,0.667403,0.040982
CosRecTfIDF_1n_3n,0.662858,0.036423
CosRecTfIDF_2n,0.574545,0.045367
CosRecTfIDF_2n_3n,0.562896,0.045939
CosRecBow_2n,0.556213,0.043927
CosRecBow_2n_3n,0.546921,0.044885


In [34]:
del models

Лучше всего срабатывает поиск ближайших по косиносному расстоянию на TFIDF с униграммами. Попробуем улучишить функцию предобработки неймингов так, чтобы она приводила литры в миллилитры и килограммы в граммы:

In [36]:
# def replace_values_l(value):
#     if 'мл' in value:
#         return value
#     elif 'л' in value:
#         pattern = r'(\d+(?:\.\d+)?)\s*л\b'  # Паттерн для нахождения числа с возможной десятичной частью и "л"
#         matches = re.findall(pattern, value, flags=re.IGNORECASE)
#         for match in matches:
#             replacement = f"{float(match) * 1000:.0f} мл"  # Корректное заменя "л" на "000 мл" с учетом десятичной части числа
#             value = re.sub(fr'({match})\s*л\b', replacement, value, flags=re.IGNORECASE)
#         return value
#     else:
#         return value

# def replace_values_kg(value):
#     if 'г' in value:
#         return value
#     elif 'кг' in value:
#         pattern = r'(\b\d+)\s*кг\b'  # Паттерн для нахождения числа и "кг"
#         matches = re.findall(pattern, value, flags=re.IGNORECASE)
#         for match in matches:
#             value = re.sub(fr'(\b{match})\s*кг\b', r'\1 1000 г', value, flags=re.IGNORECASE)  # Замена "кг" на "1000 г"
#         value = value.replace('.0', '')  # Удаление .0 при замене килограмм на граммы
#         return value
#     else:
#         return value


# переделали функции, с новыми почти на всех моделях mrr увеличился значительно
def replace_values_l(value):
    if ' л' in value:
        value = value.replace(' л', '000 мл')
        value = value.replace('.0', '')  # Удаление .0 при замене литров на миллилитры
        return value
    elif 'л' in value:
        pattern = r'(\d+(?:\.\d+)?)\s*л\b'  # Паттерн для нахождения числа с возможной десятичной частью и "л"
        matches = re.findall(pattern, value, flags=re.IGNORECASE)
        for match in matches:
            replacement = f"{float(match) * 1000:.0f} мл"  # Корректное заменя "л" на "000 мл" с учетом десятичной части числа
            value = re.sub(fr'({match})\s*л\b', replacement, value, flags=re.IGNORECASE)
        return value
    else:
        return value

def replace_values_kg(value):
    if ' кг' in value:
        value = value.replace(' кг', '000 г')
        value = value.replace('.0', '')  # Удаление .0 при замене литров на миллилитры
        return value
    elif 'кг' in value:
        pattern = r'(\d+(?:\.\d+)?)\s*кг\b'  # Паттерн для нахождения числа с возможной десятичной частью и "л"
        matches = re.findall(pattern, value, flags=re.IGNORECASE)
        for match in matches:
            replacement = f"{float(match) * 1000:.0f} г"  # Корректное заменя "л" на "000 мл" с учетом десятичной части числа
            value = re.sub(fr'({match})\s*кг\b', replacement, value, flags=re.IGNORECASE)
        return value
    else:
        return value

def string_filter_v2(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer):
    
    string = string.lower() 
    #приводим к единой системе измерения
    string = replace_values_kg(replace_values_l(string))
    #убираем лишние символы
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    #добавляем пробелы между слитыми русскими и английскими словами вроде 'стенMultipower'
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    #добавляем пробелы между слитыми словами и числами '5литров' или '900ml'
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    #удаляем английские и русские стоп-слова и стемматизируем
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])

    return string

In [37]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2)
}

In [38]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [39]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name + '_filter_v2'])])

5it [00:02,  2.09it/s]
5it [00:02,  2.10it/s]
5it [00:02,  2.17it/s]
5it [00:02,  2.10it/s]
5it [00:02,  2.14it/s]
5it [00:02,  2.18it/s]
5it [00:02,  2.13it/s]
5it [00:02,  2.10it/s]
5it [00:02,  2.17it/s]
5it [00:02,  2.15it/s]
5it [00:02,  2.12it/s]
5it [00:02,  2.18it/s]


In [40]:
results_table.sort_values('MRR', ascending=False)

,MRR,std
CosRecBow_1n_filter_v2,0.789506,0.031924
CosRecBow_1n_2n_filter_v2,0.789463,0.034928
CosRecTfIDF_1n_2n_filter_v2,0.778531,0.044540
CosRecTfIDF_1n_filter_v2,0.775225,0.035361
CosRecBow_1n_3n_filter_v2,0.774542,0.038353
CosRecTfIDF_1n_3n_filter_v2,0.769306,0.044045
CosRecTfIDF_2n_filter_v2,0.698272,0.053655
CosRecTfIDF_1n,0.696339,0.037047
CosRecBow_2n_filter_v2,0.693406,0.050227
CosRecTfIDF_1n_2n,0.679717,0.038619


Метрика подскочила, это хорошо. Теперь лидирует мешок слов со стандартными весами. Попробуем разбивать слова текста на пары или тройки букв и строить мешки из них. Разумеется, надо увеличивать количество n-грамм в мешках: 

In [41]:
def generate_ngrams(text, n):
    ngrams = [text[i:i + n] for i in range(len(text) - n + 1)]
    return ngrams

def string_filter_v3(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer,
                  ngram_len=2):
    
    string = string.lower() 
    #приводим к единой системе измерения
    string = replace_values_kg(replace_values_l(string))
    #убираем лишние символы
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    #добавляем пробелы между слитыми русскими и английскими словами вроде 'стенMultipower'
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    #добавляем пробелы между слитыми словами и числами '5литров' или '900ml'
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    #удаляем английские и русские стоп-слова и стемматизируем
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])
    #Разделение токенов-слов на уникальные тройки символов
    string = ' '.join([' '.join(generate_ngrams(word, ngram_len)) if (len(word) >= ngram_len and not word.isdigit()) else word for word in string.split()])
    return string

string = dealerprice['product_name'][0]
print(string)
print(string_filter_v3(string))

Краска-грунт фасадная для плит OSB Proff 3 в 1 Liquid Rubber 7 кг Prosept. Уцененный товар
кр ра ас ск ка аг гр ру ун нт фа ас са ад дн пл ли ит os sb pr ro of ff 3 1 li iq qu ui id ru ub bb be er 7000 г pr ro os se ep pt уц це ен не ен то ов ва ар


In [42]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_1n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_2n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_3n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(4,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_1n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_2n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_3n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(4,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
}

In [43]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [44]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name + '_filter_v3'])])

5it [00:02,  1.99it/s]
5it [00:02,  1.98it/s]
5it [00:02,  2.02it/s]
5it [00:02,  1.88it/s]
5it [00:02,  1.96it/s]
5it [00:02,  2.03it/s]
5it [00:02,  1.77it/s]
5it [00:02,  1.86it/s]
5it [00:02,  1.99it/s]
5it [00:02,  2.05it/s]
5it [00:02,  2.01it/s]
5it [00:02,  1.88it/s]
5it [00:02,  2.02it/s]
5it [00:02,  1.86it/s]
5it [00:02,  1.89it/s]
5it [00:02,  2.01it/s]
5it [00:02,  1.85it/s]
5it [00:02,  1.91it/s]
5it [00:02,  1.94it/s]
5it [00:02,  2.03it/s]


In [45]:
results_table.sort_values('MRR', ascending=False)

,MRR,std
CosRecBow_1n_filter_v2,0.789506,0.031924
CosRecBow_1n_2n_filter_v2,0.789463,0.034928
CosRecTfIDF_1n_2n_filter_v2,0.778531,0.044540
CosRecTfIDF_1n_filter_v2,0.775225,0.035361
CosRecBow_1n_3n_filter_v2,0.774542,0.038353
CosRecTfIDF_1n_2n_filter_v3,0.770999,0.039900
CosRecTfIDF_1n_3n_filter_v2,0.769306,0.044045
CosRecTfIDF_1n_filter_v3,0.766448,0.041673
CosRecTfIDF_1n_3n_filter_v3,0.759610,0.041305
CosRecTfIDF_1n_4n_filter_v3,0.751242,0.044257


Результаты не улучшились. Попробуем добавить в предобработку транслитизацию. Сначала затестим без разбиения на пары символов:

In [48]:
def string_filter_v2_t(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer,
                  transliterator=translit):
    
    string = string.lower() 
    #приводим к единой системе измерения
    string = replace_values_kg(replace_values_l(string))
    #убираем лишние символы
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    #добавляем пробелы между слитыми русскими и английскими словами вроде 'стенMultipower'
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    #добавляем пробелы между слитыми словами и числами '5литров' или '900ml'
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    #удаляем английские и русские стоп-слова и стемматизируем
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])
    #добавляем транслитирацию
    string = transliterator(string, 'ru', reversed=False)
    return string

In [49]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t)
}

In [50]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [51]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name + '_filter_v2_t'])])

5it [00:02,  2.02it/s]
5it [00:02,  2.03it/s]
5it [00:02,  2.04it/s]
5it [00:02,  2.01it/s]
5it [00:02,  2.06it/s]
5it [00:02,  2.09it/s]
5it [00:02,  2.05it/s]
5it [00:02,  2.00it/s]
5it [00:02,  2.07it/s]
5it [00:02,  2.00it/s]
5it [00:02,  2.04it/s]
5it [00:02,  2.02it/s]


In [52]:
results_table.sort_values('MRR', ascending=False)

,MRR,std
CosRecBow_1n_2n_filter_v2_t,0.791790,0.040309
CosRecBow_1n_filter_v2_t,0.790947,0.036201
CosRecBow_1n_filter_v2,0.789506,0.031924
CosRecBow_1n_2n_filter_v2,0.789463,0.034928
CosRecTfIDF_1n_2n_filter_v2_t,0.779901,0.043437
CosRecTfIDF_1n_2n_filter_v2,0.778531,0.044540
CosRecBow_1n_3n_filter_v2_t,0.778352,0.037846
CosRecTfIDF_1n_filter_v2_t,0.776851,0.037489
CosRecTfIDF_1n_filter_v2,0.775225,0.035361
CosRecBow_1n_3n_filter_v2,0.774542,0.038353


Результаты улучшились Теперь с транслитизацие и разбиением на пары символов:

In [53]:
def string_filter_v3_t(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer,
                  ngram_len=2,
                  transliterator=translit):
    
    string = string.lower() 
    #приводим к единой системе измерения
    string = replace_values_kg(replace_values_l(string))
    #убираем лишние символы
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    #добавляем пробелы между слитыми русскими и английскими словами вроде 'стенMultipower'
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    #добавляем пробелы между слитыми словами и числами '5литров' или '900ml'
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    #удаляем английские и русские стоп-слова и стемматизируем
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])
    #Разделение токенов-слов на уникальные тройки символов
    string = ' '.join([' '.join(generate_ngrams(word, ngram_len)) if (len(word) >= ngram_len and not word.isdigit()) else word for word in string.split()])
    #добавляем транслитирацию
    string = transliterator(string, 'ru')
    return string

In [54]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_1n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_2n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_3n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(4,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_1n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_2n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_3n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(4,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
}

In [55]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [56]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name + '_filter_v3_t'])])

5it [00:02,  1.87it/s]
5it [00:02,  1.81it/s]
5it [00:02,  1.86it/s]
5it [00:02,  1.77it/s]
5it [00:02,  1.85it/s]
5it [00:02,  1.87it/s]
5it [00:02,  1.72it/s]
5it [00:02,  1.81it/s]
5it [00:02,  1.87it/s]
5it [00:02,  1.89it/s]
5it [00:02,  1.89it/s]
5it [00:02,  1.87it/s]
5it [00:02,  1.85it/s]
5it [00:02,  1.74it/s]
5it [00:02,  1.86it/s]
5it [00:02,  1.94it/s]
5it [00:02,  1.73it/s]
5it [00:02,  1.78it/s]
5it [00:02,  1.86it/s]
5it [00:02,  1.94it/s]


In [57]:
results_table.sort_values('MRR', ascending=False)

,MRR,std
CosRecBow_1n_2n_filter_v2_t,0.791790,0.040309
CosRecBow_1n_filter_v2_t,0.790947,0.036201
CosRecBow_1n_filter_v2,0.789506,0.031924
CosRecBow_1n_2n_filter_v2,0.789463,0.034928
CosRecTfIDF_1n_2n_filter_v2_t,0.779901,0.043437
...,...,...
CosRecTfIDF_3n_filter_v2,0.403145,0.081706
CosRecBow_3n_filter_v2_t,0.395048,0.078241
CosRecBow_3n_filter_v2,0.393280,0.080280
CosRecTfIDF_3n,0.359115,0.065103


Транслитирация немного улучшает результаты, а разбиение токенов на пары символов - наоборот. Похоже что мы выжали максимум из мешков слов, а значит двигаемся к плотным векторным представлениям:

In [64]:
def string_filter_emb(string):
    
    string = string.lower() 
    #приводим к единой системе измерения
    string = replace_values_kg(replace_values_l(string))
    #добавляем пробелы между слитыми русскими и английскими словами вроде 'стенMultipower'
    string = re.sub(r'[^a-zo0-9а-я\s:]', ' ', string)
    #добавляем пробелы между слитыми русскими и английскими словами вроде 'стенMultipower'
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    #добавляем пробелы между слитыми словами и числами '5литров' или '900ml'
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    return string

string = dealerprice['product_name'][0]
print(string)
print(string_filter_emb(string))

Краска-грунт фасадная для плит OSB Proff 3 в 1 Liquid Rubber 7 кг Prosept. Уцененный товар
краска грунт фасадная для плит osb proff 3 в 1 liquid rubber 7000 г prosept  уцененный товар


In [66]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
bert = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

Оборачиваем sbert, чтобы работал как обычный векторайзер sklearn (чтобы корректно встал в DistanceRecommender). Далее аналогичные действия для других моделей не комментируются ^^

In [67]:
class SbertVectorizer():
    
    def __init__(self,
                 tokenizer=tokenizer,
                 model=bert):
        
        self.tokenizer = tokenizer
        self.model = model
    
    def fit(self, X=None):        
        pass
    

    
    def transform(self, corpus):
        encoded_input = tokenizer(corpus, padding=True, truncation=True, max_length=24, return_tensors='pt')
        
        with torch.no_grad():
            model_output = self.model(**encoded_input) 
        
        token_embeddings = model_output[0]
        input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        
        sentence_embeddings = sum_embeddings / sum_mask
        
        return sentence_embeddings.numpy()

In [68]:
models = {'CosEmb_sbert': DistanceRecommender(vectorizer=SbertVectorizer(tokenizer=tokenizer, model=bert), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [69]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [70]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name])])

5it [03:20, 40.01s/it]


In [71]:
results_table.loc['CosEmb_sbert', :]

MRR    0.457132
std    0.013280
Name: CosEmb_sbert, dtype: float64

In [72]:
del tokenizer, bert

Наблюдаем низкий MRR, а это означает что модель нам не подходит. Вероятно проблема в том, что sbert корректно работает только с русскими токенами, а у нас в неймингах присутствуют английские.

In [73]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [74]:
class BertVectorizer():
    
    def __init__(self,
                 tokenizer=tokenizer,
                 model=bert):
        
        self.tokenizer = tokenizer
        self.model = model
    
    def fit(self, X=None):        
        pass
    

    
    def transform(self, text):
        t = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**{k: v.to(self.model.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings.numpy()

In [75]:
models = {'RubertTiny2_Cos': DistanceRecommender(vectorizer=BertVectorizer(tokenizer=tokenizer, model=bert), simularity_func=cosine_similarity, text_prep_func=string_filter_emb,)}

In [76]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [77]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name])])

5it [00:04,  1.15it/s]


In [78]:
results_table.loc[models.keys()]

,MRR,std
RubertTiny2_Cos,0.698623,0.057061


In [79]:
del tokenizer, bert

Наконец-то более-менее конкурентноспособные результаты от bert-like модели. Посмотрим реализацию sbert_large_mt_nlu_ru:

In [80]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
bert = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

In [81]:
models = {'CosEmb_mt_sbert': DistanceRecommender(vectorizer=SbertVectorizer(tokenizer=tokenizer, model=bert), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [82]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [83]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name])])

5it [03:18, 39.60s/it]


In [84]:
results_table.loc[models.keys()]

,MRR,std
CosEmb_mt_sbert,0.532075,0.014792


In [85]:
del bert, tokenizer

No comments, двигаемся дальше. Теперь можно посмотреть берты работающие с несколькими языками, одним из лучших считаются LaBSE. Для начала глянем урезанную LaBSE, работающую только с русскими и английскими токенами:

In [86]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
bert = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

In [87]:
models = {'CosEmb_LaBSE-en-ru': DistanceRecommender(vectorizer=BertVectorizer(tokenizer=tokenizer, model=bert), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [88]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [89]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name])])

5it [01:15, 15.18s/it]


In [90]:
results_table.loc[models.keys()]

,MRR,std
CosEmb_LaBSE-en-ru,0.73808,0.049925


In [91]:
del bert, tokenizer

Метрика лучше чем у предыдущих бертов. Теперь посмотрим большую Labse. Приятно, что у неё есть обёртка SentenceTransformer.

In [94]:
transformer = SentenceTransformer('sentence-transformers/LaBSE')

In [95]:
class TransformerVectorizer():
    
    def __init__(self,
                 transformer=transformer):
        
        self.transformer = transformer
    
    def fit(self, X=None):        
        pass
    
    def transform(self, corpus):
        embeddings = transformer.encode(corpus) 
        return embeddings

In [96]:
models = {'CosEmb_LaBSE': DistanceRecommender(vectorizer=TransformerVectorizer(transformer=transformer), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [97]:
for model in models.values():
    model.fit(product, 'name', 'id')

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [98]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name])])

0it [00:00, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

1it [00:13, 13.04s/it]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

2it [00:24, 12.27s/it]

...

3it [00:37, 12.47s/it]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

4it [00:50, 12.56s/it]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

5it [01:03, 12.61s/it]


In [99]:
results_table.loc[models.keys()]

,MRR,std
CosEmb_LaBSE,0.765149,0.042544


In [100]:
del transformer

Продвинулись дальше, но пока не догнали мешки с n-граммами. Напоследок посмотрим state of the art векторайзер на русских текстах:

In [102]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
vectorizer = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

class InfloatVectorizer():
    
    def __init__(self,
                 tokenizer=tokenizer,
                 vectorizer=vectorizer):
        
        self.tokenizer = tokenizer
        self.model = vectorizer
    
    def fit(self, X=None):        
        pass
    

    
    def transform(self, corpus):
        batch_dict = self.tokenizer(corpus, max_length=512, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.model(**batch_dict)
        last_hidden = outputs.last_hidden_state.masked_fill(~batch_dict['attention_mask'][..., None].bool(), 0.0)
        embeddings = last_hidden.sum(dim=1) / batch_dict['attention_mask'].sum(dim=1)[..., None]
        embeddings = F.normalize(embeddings, p=2, dim=1)
        return embeddings.numpy()

In [103]:
models = {'Infloat_multilingual': DistanceRecommender(vectorizer=InfloatVectorizer(tokenizer=tokenizer, vectorizer=vectorizer), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [104]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [105]:
kf = KFold(n_splits=5)

for name, model in models.items():
    results = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice, left_on='key', right_on='product_key', how='inner')
        
        true_ids = test_interactions['product_id'].values
        
        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results), 'std': np.std(results)}, index=[name])])

5it [05:23, 64.62s/it]


In [106]:
results_table.loc[models.keys()]

,MRR,std
Infloat_multilingual,0.810799,0.048534


Наконец-то удалось победить мешок с n-граммами. Хочется отметить, что помимо показанных выше bert-like моделей командой были рассмотрены и отвалидированы несколько других. 

In [107]:
del tokenizer, vectorizer

#### Результаты рассмотренных решений

In [108]:
results_table.sort_values('MRR', ascending=False).head(20)

,MRR,std
Infloat_multilingual,0.810799,0.048534
CosRecBow_1n_2n_filter_v2_t,0.791790,0.040309
CosRecBow_1n_filter_v2_t,0.790947,0.036201
CosRecBow_1n_filter_v2,0.789506,0.031924
CosRecBow_1n_2n_filter_v2,0.789463,0.034928
CosRecTfIDF_1n_2n_filter_v2_t,0.779901,0.043437
CosRecTfIDF_1n_2n_filter_v2,0.778531,0.044540
CosRecBow_1n_3n_filter_v2_t,0.778352,0.037846
CosRecTfIDF_1n_filter_v2_t,0.776851,0.037489
CosRecTfIDF_1n_filter_v2,0.775225,0.035361


для интереса глянем, сколько всего вариаций решения рассмотрели:

#### Выводы:

Лучший MRR показала модель, ранжирующая по косинусному сходству эмбеддинги xlm-roberta-large модели, предобученной командой *intfloat* (https://huggingface.co/intfloat/multilingual-e5-large). Текущее решение уже показывает отличные результаты, и вполне способно стать финальным. 
На кросс-валидации модель демонстрирует **0.81** **MRR@10**. Это означает, что **средний ранг** релевантного айтема в списке рекомендаций модели приблизительно 
равен **1.23** (чаще всего на первом месте, редко на втором, редко-редко на других). Таким образом разметчик сможет практически моментально находить релевантный айтем заказчика.

#### Отдаём backend команде:

Файлы для инициализации токенайзера и векторайзера берт-модели, матрицу векторов продуктов заказчика и словарь для метода `from_pretrained` модели, а так же python файл с функциями и классами:

*Учитывая сроки выделенные на работу дс команд, мы изначально не рассматривали возможность успеть подготовить модель второго этапа к дедлайну (19:00 29 ноября, а начали полноценно работать только 26 ноября), а потому модель первого этапа сразу делали так, чтобы она сама по себе была отличным финальным решением. Файлы для инициализации токенайзера и векторайзера берт-модели, матрицу векторов продуктов заказчика и словарь для метода `from_pretrained` модели уже переданы backend команде. Вот набор функций и объектов, которые со всем этим работают*

In [231]:
# import json
# import re

# import numpy as np
# import pandas as pd
# import torch
# import torch.nn.functional as F
# from sklearn.metrics.pairwise import cosine_similarity
# from transformers import AutoModel, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained('./tokenizer')
# vectorizer = AutoModel.from_pretrained('./vectorizer')


# class NumpyEncoder(json.JSONEncoder):
#     def default(self, obj):
#         if isinstance(obj, np.integer):
#             return int(obj)
#         return super(NumpyEncoder, self).default(obj)


# def replace_values_l(value):
#     if ' л' in value:
#         value = value.replace(' л', '000 мл')
#         value = value.replace('.0', '')
#         return value
#     elif 'л' in value:
#         pattern = r'(\d+(?:\.\d+)?)\s*л\b'
#         matches = re.findall(pattern, value, flags=re.IGNORECASE)
#         for match in matches:
#             replacement = f"{float(match) * 1000:.0f} мл"
#             value = re.sub(
#                 fr'({match})\s*л\b',
#                 replacement,
#                 value,
#                 flags=re.IGNORECASE
#             )
#         return value
#     else:
#         return value


# def replace_values_kg(value):
#     if ' кг' in value:
#         value = value.replace(' кг', '000 г')
#         value = value.replace('.0', '')
#         return value
#     elif 'кг' in value:
#         pattern = r'(\d+(?:\.\d+)?)\s*кг\b'
#         matches = re.findall(pattern, value, flags=re.IGNORECASE)
#         for match in matches:
#             replacement = f"{float(match) * 1000:.0f} г"
#             value = re.sub(
#                 fr'({match})\s*кг\b',
#                 replacement,
#                 value,
#                 flags=re.IGNORECASE
#             )
#         return value
#     else:
#         return value


# def string_filter_emb(string):
#     string = string.lower()
#     string = replace_values_kg(replace_values_l(string))
#     string = re.sub(r'[^a-zo0-9а-я\s:]', ' ', string)
#     string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
#     string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
#     return string


# class InfloatVectorizer():
#     def __init__(self,
#                  tokenizer=tokenizer,
#                  vectorizer=vectorizer):

#         self.tokenizer = tokenizer
#         self.model = vectorizer

#     def fit(self, X=None):
#         pass

#     def transform(self, corpus):
#         batch_dict = self.tokenizer(
#             corpus,
#             max_length=512,
#             padding=True,
#             truncation=True,
#             return_tensors='pt'
#         )
#         with torch.no_grad():
#             outputs = self.model(**batch_dict)
#         last_hidden = outputs.last_hidden_state.masked_fill(
#             ~batch_dict['attention_mask'][..., None].bool(), 0.0
#         )
#         embeddings = (last_hidden.sum(dim=1)
#                       / batch_dict['attention_mask'].sum(dim=1)[..., None])
#         embeddings = F.normalize(embeddings, p=2, dim=1)
#         return embeddings.numpy()


# class DistanceRecommender():
#     def __init__(self,
#                  vectorizer,
#                  simularity_func,
#                  text_prep_func):
#         self.vectorizer = vectorizer
#         self.simularity_counter = simularity_func
#         self.preprocessing = text_prep_func

#     def fit(self,
#             product_corpus,
#             name_column,
#             id_column,
#             save_to_dir=False):
#         preprocessed_corpus = (
#             product_corpus[name_column].apply(
#                 self.preprocessing
#             ).values.tolist()
#         )
#         self.vectorizer.fit(preprocessed_corpus)
#         self.product_matrix = self.vectorizer.transform(preprocessed_corpus)
#         self.product_index_to_id = {i: product_corpus.loc[i, id_column] for i in range(len(product_corpus))}
#         if save_to_dir:
#             np.save('product_matrix.npy', self.product_matrix)

#             with open('product_index_to_id.json', 'w') as file:
#                 json.dump(self.product_index_to_id, file, cls=NumpyEncoder)


#     def from_pretrained(
#         self,
#         product_matrix_path='./model_files/product_matrix.npy',
#         product_index_to_id_dict_path='./model_files/product_index_to_id.json'
#     ):
#         self.product_matrix = np.load(product_matrix_path)

#         with open(product_index_to_id_dict_path, 'rb') as file:
#             self.product_index_to_id = json.load(file)

#     def recommend(self,
#                   dealer_corpus: list[dict]):
#         preprocessed_corpus = dealer_corpus.apply(
#             self.preprocessing
#         ).values.tolist()
#         vectors = self.vectorizer.transform(preprocessed_corpus)
#         sims = self.simularity_counter(vectors, self.product_matrix)

#         result = []
#         for vec in sims:
#             result += [[self.product_index_to_id[str(index)] for index in vec.argsort()[::-1]]]
#         return np.array(result)


# def dealerprice_table(table_path='marketing_dealerprice.csv',
#                       product_id_column='product_key',
#                       dealer_id_column='dealer_id',
#                       read_params={'on_bad_lines': "skip",
#                                    'encoding': 'utf-8',
#                                    'sep': ';'}):
#     '''
#     Функция принимает:
#     .Путь к csv файлу, содержащему результаты парсинга.
#     .Названия колонок с id товаров и id дилеров
#     .Параметры чтения csv можно указать, если вдруг они изменятся.
#     '''

#     table_csv = pd.read_csv(table_path, **read_params)
#     table_csv = table_csv.sort_values(
#         'date', ascending=False
#     ).drop_duplicates(
#         subset=[
#             product_id_column,
#             dealer_id_column
#         ]
#     )
#     return table_csv


# model = DistanceRecommender(
#     vectorizer=InfloatVectorizer(
#         tokenizer=tokenizer,
#         vectorizer=vectorizer
#     ),
#     simularity_func=cosine_similarity,
#     text_prep_func=string_filter_emb
# )

Применение:

In [ ]:
# model.from_pretrained()


# names = ['Герметик акриловый  цвет белый , ф/п 600 мл. (12 штук )',
#          'Гель эконом-класса для мытья  посуды вручную. С ароматом яблокаCooky Apple Eконцентрированное средство / 5 л ПЭТ',
#          'Средство для удаления ржавчины и минеральных отложений щадящего действияBath Acid  концентрат 1:200-1:500 / 0,75 л ',
#          'Антисептик многофункциональный ФБС, ГОСТ / 5 л',
#          'Гелеобразное средство усиленного действия для удаления ржавчины и минеральных отложенийBath Extraконцентрат 1:10-1:100 / 0,75 л']

# names = pd.Series(names)

# print(model.recommend(names))